# CIS 419/519 
# Ipython Debugging
<!--BOOK_INFORMATION-->
#### Contributors: Ty Nguyen and Dan Gallagher
#### Credits: *This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; 

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.06-Errors-and-Debugging.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 1. Introduction to Errors and Debugging

Code development and data analysis always require a bit of trial and error, and IPython contains tools to streamline this process.
This section will briefly cover some options for controlling Python's exception reporting, followed by exploring tools for debugging errors in code.

## 1.1 Controlling Exceptions: ``%xmode``

Most of the time when a Python script fails, it will raise an Exception.
When the interpreter hits one of these exceptions, information about the cause of the error can be found in the *traceback*, which can be accessed from within Python.
With the ``%xmode`` magic function, IPython allows you to control the amount of information printed when the exception is raised.
Consider the following code:

In [1]:
def func1(a, b):
    b -= 1
    return a / b

def func2(x):
    a = x
    b = x - 1
    return func1(a, b)

In [2]:
func2(2)

ZeroDivisionError: ignored

Calling ``func2`` results in an error, and reading the printed trace lets us see exactly what happened.
By default, this trace includes several lines showing the context of each step that led to the error.
Using the ``%xmode`` magic function (short for *Exception mode*), we can change what information is printed.

``%xmode`` takes a single argument, the mode, and there are three possibilities: ``Plain``, ``Context``, and ``Verbose``.
The default is ``Context``, and gives output like that just shown before.
``Plain`` is more compact and gives less information:

In [3]:
%xmode Plain

Exception reporting mode: Plain


In [5]:
func2(2)

ZeroDivisionError: ignored

The ``Verbose`` mode adds some extra information, including the arguments to any functions that are called:

In [6]:
%xmode Verbose

Exception reporting mode: Verbose


In [7]:
func2(2)

ZeroDivisionError: ignored

This extra information can help narrow-in on why the exception is being raised.
So why not use the ``Verbose`` mode all the time?
As code gets complicated, this kind of traceback can get extremely long.
Depending on the context, sometimes the brevity of ``Default`` mode is easier to work with.

## 1.2 Debugging: When Reading Tracebacks Is Not Enough

The standard Python tool for interactive debugging is ``pdb``, the Python debugger.
This debugger lets the user step through the code line by line in order to see what might be causing a more difficult error.
The IPython-enhanced version of this is ``ipdb``, the IPython debugger.

There are many ways to launch and use both these debuggers; we won't cover them fully here.
Refer to the online documentation of these two utilities to learn more.

In IPython, perhaps the most convenient interface to debugging is the ``%debug`` magic command.
If you call it after hitting an exception, it will automatically open an interactive debugging prompt at the point of the exception.
The ``ipdb`` prompt lets you explore the current state of the stack, explore the available variables, and even run Python commands!

Let's look at the most recent exception, then do some basic tasks–print the values of ``a`` and ``b``, and type ``quit`` to quit the debugging session:

In [8]:
%debug

> <ipython-input-1-69df7d0437bc>(3)func1()
      1 def func1(a, b):
      2     b -= 1
----> 3     return a / b
      4 
      5 def func2(x):

ipdb> print(b)
0
ipdb> print(a)
2
ipdb> print(a/b)
*** ZeroDivisionError: division by zero
ipdb> q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



The interactive debugger allows much more than this, though– we can even step up and down through the stack and explore the values of variables there:

In [9]:
%debug


> <ipython-input-1-69df7d0437bc>(3)func1()
      1 def func1(a, b):
      2     b -= 1
----> 3     return a / b
      4 
      5 def func2(x):

ipdb> u
> <ipython-input-1-69df7d0437bc>(8)func2()
      4 
      5 def func2(x):
      6     a = x
      7     b = x - 1
----> 8     return func1(a, b)

ipdb> print(b)
1
ipdb> d
> <ipython-input-1-69df7d0437bc>(3)func1()
      1 def func1(a, b):
      2     b -= 1
----> 3     return a / b
      4 
      5 def func2(x):

ipdb> print(b)
0
ipdb> q


This allows you to quickly find out not only what caused the error, but what function calls led up to the error.

If you'd like the debugger to launch automatically whenever an exception is raised, you can use the ``%pdb`` magic function to turn on this automatic behavior:

In [ ]:
%xmode Plain
%pdb on
# to turn it off again, we just use % pdb off
func2(2)

Finally, if you have a script that you'd like to run from the beginning in interactive mode, you can run it with the command ``%run -d``, and use the ``next`` command to step through the lines of code interactively.


---
## 1.3 Partial list of debugging commands

There are many more available commands for interactive debugging than we've listed here; the following table contains a description of some of the more common and useful ones:

| Command         |  Description                                                |
|-----------------|-------------------------------------------------------------|
| ``list``        | Show the current location in the file                       |
| ``h(elp)``      | Show a list of commands, or find help on a specific command |
| ``q(uit)``      | Quit the debugger and the program                           |
| ``c(ontinue)``  | Quit the debugger, continue in the program                  |
| ``n(ext)``      | Go to the next step of the program                          |
| ``<enter>``     | Repeat the previous command                                 |
| ``p(rint)``     | Print variables                                             |
| ``s(tep)``      | Step into a subroutine                                      |
| ``r(eturn)``    | Return out of a subroutine                                  |

More details: \\
``n[ext]`` : n simply continues program execution to the next line in the current method

``s[tep]`` : s steps to the very next line of executable code, whether it is inside a called method or just on the next line

On a line where a method is being called, n will “step over” the method execution code while s will “step into” the method execution code allowing you to introspect the method code.

``w[here]`` : w prints a stack trace, with the most recent frame at the bottom. An arrow indicates the “current frame”, which determines the context of most commonds.

``b[reak] ([filename:]lineno|function)`` : b ddds breakpoints to the specified locations. Example usage:

``b sample-filename.py:<line no>``
``b <function>``
``b <lineno> (for the current file)``

``c[ontinue]`` : c continues program execution until another breakpoint is hit or the program execution completes

``a[rgs]`` : a prints out all the arguments the current function received

``r[eturn]`` : r continues execution until the current function returns

In case you are already using variables with names such as c, a use complete command continue, args to get desired operation.

For more information, use the ``help`` command in the debugger, or take a look at ``ipdb``'s [online documentation](https://github.com/gotcha/ipdb).

---

## 1.4 Setting Breakpoints

Sometimes, we know we'll want to check values in specific places. At some point or another, I suspect most of us might have used a print() statement in this case. But we can actually tell pdb to stop at certain places by using the pdb.set_trace() function.

In Python 3.7 and later, a built in breakpoint() function was made with similar functionality, but with the ability to use an environmental BREAKPOINT variable that can enable or disable all breakpoints. It is recommended to use this option if possible, but it can sometimes cause Colab cells to crash so for this tutorial we will stick with set_trace()

Let's revisit our simple functions and add a new one to test this out

In [11]:
%pdb on

# to use set_trace, we have to actively import pdb
import pdb

def func1(a, b):
    b -= 1
    return a / b

def func2(x):
    a = x
    b = x - 1
    return func1(a, b)

def func3(y):
    return y*2

a = 0
pdb.set_trace()
a = func3(2)
pdb.set_trace()
func2(a-2)

Automatic pdb calling has been turned ON
--Return--
[23] > <ipython-input-11-ccb8a8207648>(19)<module>()->None
-> pdb.set_trace()
(Pdb++) c
--Return--
[23] > <ipython-input-11-ccb8a8207648>(21)<module>()->None
-> pdb.set_trace()
(Pdb++) 


ZeroDivisionError: ignored

> <ipython-input-11-ccb8a8207648>(8)func1()
      6 def func1(a, b):
      7     b -= 1
----> 8     return a / b
      9 
     10 def func2(x):

ipdb> a
2
ipdb> args
a = 2
b = 0
ipdb> q


Sometimes, we want to set a trace in a loop to check on how different values are changing over time. However, if we our looping hundreds of times, it is intractible to finish the loop with a new breakpoint each time. What if we want to see how values change up to a certain point, then let it run on its own?

Manually added breakpoints and those added with the breakpoint() function allow you to use commands like clear, disable, and ignore to accomplish this (refer to the [documentation](https://docs.python.org/3/library/pdb.html) for more info). But if you are using pdb.set_trace(), it's not quite as easy. Luckily, like many problems in Python, there is an open source fix.

We can instal pdb ++ by running pip instal pdbpp, which will overwrite our pdb module and act mostly the same, but add some nice quality of life upgrades, like the ability to disable and enable our breakpoints from within the debugger. We encourage you to check out the [pdb++ documentation](https://pypi.org/project/pdbpp/) other upgrades!

In [ ]:
pip install pdbpp

     |████████████████████████████████| 48 kB 2.9 MB/s 
  Created wheel for pyrepl: filename=pyrepl-0.9.0-py3-none-any.whl size=59907 sha256=b55d26e68856e38d3bce746e18e28a9a7482ef99d0d8887ee503e9da4fe65f9c
  Stored in directory: /root/.cache/pip/wheels/3c/d7/37/7503eea89bb7987b7553bd90528a3554e3be0dd90b095c782c
  Created wheel for wmctrl: filename=wmctrl-0.4-py3-none-any.whl size=3857 sha256=6854ac748f4764bed2327d64bc71df12ed31db5a5b6b4fea02d4412e802c5621
  Stored in directory: /root/.cache/pip/wheels/34/41/26/7eb2a9e48d9024b4607b7f684c55c9ce3a662e17b7a8b50e67
Successfully built pyrepl wmctrl


In [12]:
# We'll turn the auto pdb off again for now
%pdb off
pdb.enable()

import pdb

x = 1
y = 2
for i in range(10):
  x*=2
  y*=3
  z = x*y
  pdb.set_trace()

Automatic pdb calling has been turned OFF
[23] > <ipython-input-12-8f120056ae80>(9)<module>()->None
-> for i in range(10):
(Pdb++) x
2
(Pdb++) y
6
(Pdb++) z
12
(Pdb++) c
[23] > <ipython-input-12-8f120056ae80>(9)<module>()->None
-> for i in range(10):
(Pdb++) x
4
(Pdb++) y
18
(Pdb++) z
72
(Pdb++) pdb.disable()
(Pdb++) c


In [13]:
pdb.enable()

# In class Practice

Now that we've gotten the hang of some debugging commands, let's go over a simple practice problem. In this problem, we are attempting to create a function that, for a few matrix sizes, creates a matrix of euclidean distances from a coordinate made of each cell's indices to the origin (0,0).

We have some starter code, but it needs a fair amount of debugging!

In [24]:
#TODO: Debug this code!
import numpy as np

def getDistfromOrig(x):
  pdb.set_trace()
  return np.linalg.norm(x,2) 
    
def assignDist2Mat(matrix, rows, cols):
  """
  @Brief: Given a 2D matrix whose each cell has row and column indexes representing 
      its coordinate in the Euclidean coordinate system. This function calculates the 
      Euclidean distance from the origin ([0,0]) to every cell and assigns this value
      to the cell. 
  @Input:
    matrix (List(List)): 2D matrix 
  """
  

  for i in range(rows):
    for j in range(cols):
      # Get distance from a cell to the origin (i.e: [0,0])  
      matrix[i][j] = getDistfromOrig([i,j])
  pdb.set_trace()

def main():
  for iter in range(3):
    rows, cols = 5 + iter, 10 + iter
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    assignDist2Mat(matrix, rows, cols)
    
#TODO: automatically start the debugger after an unhandled exception
%pdb on
#TODO: set the exception mode to Verbose
%xmode Verbose
main()


Automatic pdb calling has been turned ON
Exception reporting mode: Verbose
[26] > <ipython-input-24-f7c6594dfa80>(6)getDistfromOrig()
-> return np.linalg.norm(x,2)
(Pdb++) x
[0, 0]
(Pdb++) c
[26] > <ipython-input-24-f7c6594dfa80>(6)getDistfromOrig()
-> return np.linalg.norm(x,2)
(Pdb++) x
[0, 1]
(Pdb++) q


BdbQuit: ignored

> /usr/lib/python3.7/bdb.py(113)dispatch_line()
    111         if self.stop_here(frame) or self.break_here(frame):
    112             self.user_line(frame)
--> 113             if self.quitting: raise BdbQuit
    114         return self.trace_dispatch
    115 

ipdb> q


# 2. Take-home Practice

Here's a problem to practice debugging on your own. An incorrect solution is up first that needs debugging, and the correct solution is right below. See if you can use the tools you learned today to debug this code! (Note: this is purely practice- we won't be going over this problem in class)

## 2.1 Problem: [Longest Increasing Path in a Matrix](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/) 

Given an ```m x n ``` integers matrix, return the length of the longest increasing path in matrix.

From each cell, you can either move in four directions: left, right, up, or down. You may not move diagonally or move outside the boundary (i.e., wrap-around is not allowed).

Example 1: \\
Input: matrix = [[9,9,4],[6,6,8],[2,1,1]] \\
Output: 4

![Image](https://assets.leetcode.com/uploads/2021/01/05/grid1.jpg)


Explanation: The longest increasing path is [1, 2, 6, 9].



## 2.2 Solution with Bugs

In [ ]:
def longestIncreasingPath(matrix):
    """
    @Inputs: 
      matrix (List[List]): given 2D-array
    @Outputs:
      (int): value of the longest increasing path
    """
    directions = [(0,1), (1,0), (0,-1), (-1,0.0)]
    def dfs(row = 0, col = 0, visited=[]):
        nonlocal rows, cols
        if mem[row][col]:
            return  mem[row][col]
        # Mark the current node as visited        
        visited[row][col] = 1
        
        # Find the maximum len of remaining path
        max_remain_len = 0
        # Consider 4 directions
        for dr, dc in directions: # (0,-1)
            nr, nc = row + dr, col + dc # (2, 0) 

            # Only visit this next node if it's not out of the boundary, not visited, and contains a higher value then the current node
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc] and matrix[nr][nc] > matrix[row][col]:
                max_remain_len = max(max_remain_len, dfs(nr, nc))
      
        # Backtrack
        visited[row][col] = 0
        mem[row][col] = max_remain_len + 1 
        return max_remain_len + 1
  
    if len(matrix) == 0:
        return 0 
    
    rows, cols = len(matrix), len(matrix[0])
    max_len = 0
    visited = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # Memory to store subsolutions
    mem = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # Iterate through all nodes, get the maximum increasing len
    for i in range(rows):
        for j in range(cols):  
            cur_max_len = dfs(row = i, col = j)
            max_len = max(max_len, cur_max_len)
  
    return max_len

In [ ]:
%pdb on 
%xmode Verbose 
matrix = [[9,9,4],[6,6,8],[2,1,1]]
max_len = longestIncreasingPath(matrix)
if max_len == 4:
  print("Correct answer!")
else:
  print("Incorrect answer!")

## 2.3 Correct Solution

In [ ]:
#@title
def longestIncreasingPath(matrix):
    """
    @Inputs: 
      matrix (List[List]): given 2D-array
    @Outputs:
      (int): value of the longest increasing path
    """
    directions = [(0,1), (1,0), (0,-1), (-1,0)]
    def dfs(row = 0, col = 0, visited=[]):
        nonlocal rows, cols
        if mem[row][col]:
            return  mem[row][col]
        # Mark the current node as visited        
        visited[row][col] = 1
        
        # Find the maximum len of remaining path
        max_remain_len = 0
        # Consider 4 directions
        for dr, dc in directions: # (0,-1)
            nr, nc = row + dr, col + dc # (2, 0) 

            # Only visit this next node if it's not out of the boundary, not visited, and contains a higher value then the current node
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc] and matrix[nr][nc] > matrix[row][col]:
                max_remain_len = max(max_remain_len, dfs(nr, nc, visited))
      
        # Backtrack
        visited[row][col] = 0
        mem[row][col] = max_remain_len + 1 
        return max_remain_len + 1
  
    if len(matrix) == 0:
        return 0 
    
    rows, cols = len(matrix), len(matrix[0])
    max_len = 0
    visited = [[0 for _ in range(cols)] for _ in range(rows)]
    mem = [[0 for _ in range(cols)] for _ in range(rows)]
    
    # Iterate through all nodes, get the maximum increasing len
    for i in range(rows):
        for j in range(cols): # i=2,j=1
            cur_max_len = dfs(row = i, col = j, visited=visited)
            max_len = max(max_len, cur_max_len)
  
    return max_len


matrix = [[9,9,4],[6,6,8],[2,1,1]]
max_len = longestIncreasingPath(matrix)
if max_len == 4:
  print("Correct answer!")
else:
  print("Incorrect answer!")

<!--NAVIGATION-->
< [IPython and Shell Commands](01.05-IPython-And-Shell-Commands.ipynb) | [Contents](Index.ipynb) | [Profiling and Timing Code](01.07-Timing-and-Profiling.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.06-Errors-and-Debugging.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
